<a href="https://colab.research.google.com/github/AlyShmahell/WeirdTargets/blob/master/weirdtargets2020.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [ ]:
! pip install ujson

In [ ]:
import os
import gc
import sys
import ujson
import psutil
import requests
import numpy     as     np
import pandas    as     pd
import operator  as     op
from   copy      import copy
from   tqdm      import tqdm
from   functools import reduce
from   gzip      import GzipFile
from   toolz     import partition_all
from   itertools import islice
from   pathlib   import Path
from   concurrent.futures import ProcessPoolExecutor, ThreadPoolExecuter

In [ ]:
a = partition_all(2, partition_all(2, range(10)))
b = islice(a, 2, 4)
c = islice(a, 4, 6)
print([*a], [*b], [*c])

[((0, 1), (2, 3)), ((4, 5), (6, 7)), ((8, 9),)] [] []


In [ ]:
class System:
    memory = int(np.floor(psutil.virtual_memory()[1]/(1024**3))*int(2e+4))
    cpu    = os.cpu_count()
    @classmethod
    def rmtree(root):
        for p in root.iterdir():
            if p.is_dir():
                rmtree(p)
            else:
                p.unlink()
        root.rmdir()

In [ ]:
s = System()
print(s)

In [ ]:
System.memory

220000

# nCr

In [ ]:
def ncr(n, r):
    r     = min(r, n-r)
    numer = reduce(op.mul, range(n, n-r, -1), 1)
    denom = reduce(op.mul, range(1, r+1), 1)
    return  numer / denom

# Downloader

In [ ]:
def downloader(url):
    def func(r, path, filename, total_size, block_size):
        t          = tqdm(
            desc       = f"downloading {filename}",
            total      = total_size, 
            unit       = 'iB', 
            unit_scale = True
        )
        with open(path/filename, 'wb') as f:
            for data in r.iter_content(block_size):
                t.update(len(data))
                f.write(data)
        t.close()
        try:
            assert not(total_size != 0 and t.n != total_size)
        except:
            sys.exit(f"downloaded {t.n}/{total_size}", flush=True)
    r          = requests.get(url, stream=True)
    total_size = int(r.headers.get('content-length', 0))
    block_size = 1024**2
    filename   = url.split('/')[-1]
    path       = Path(Path.home())/Path('workspace')
    path.mkdir(parents=True, exist_ok=True)
    try:
        assert Path(path/filename).stat().st_size == total_size
        print(f"{filename} is already downloaded", flush=True)
    except:
        func(r, path, filename, total_size, block_size)
    filename    = filename.split('.')
    (filename, 
     extension) = ".".join(filename[:-1]), filename[-1]
    return path, filename, extension

In [ ]:
df = pd.DataFrame({
    'a': [1,2,3],
    'b': [4,5,6]
})
len(df.index)

3

In [ ]:
class Prerocessor:
    def __traverse__(self, parsed, keys):
        if len(keys)>1:
            return self.__traverse__(parsed[keys[0]], keys[1:])
        return parsed[keys[0]]
    def __json__(self, string):
        parsed = ujson.loads(string)
        obj = {
            key: self.__traverse__(parsed, self.keygroup[key])
            for key in self.keygroup.keys()
        }
        return obj
    def __df__(self, batch):
        json = map(lambda string: self.__json__(string), batch)
        df   = pd.DataFrame.from_records(json, 
                                         columns=["target",
                                                  "disease",
                                                  "score"]
        )
        return df
    def __init__(self, url, keygroup):
        (path, 
         filename, 
         extension)   = downloader(url)
        self.keygroup = keygroup
        with tqdm(desc=f"Feature Extraction") as tqdmo:
            with GzipFile(f"{path}/{filename}.{extension}") as f:
                batches = partition_all(System.memory, f)
                for batch in batches:
                    df = self.__df__(batch)
                    df.to_hdf(
                        f"{path}/{filename}.hdf",
                        key='features',
                        mode='a',
                        format='table',
                        append=True
                    )
                    tqdmo.update(len(df.index))
        self.df = pd.read_hdf(
                f"{path}/{filename}.hdf",
                key='features'
            )
        print(self.df.head(5))


In [ ]:
%timeit
Prerocessor(
    'https://storage.googleapis.com/open-targets-data-releases/17.12/17.12_evidence_data.json.gz',
    {
        "target" : ["target", "id"],
        "disease": ["disease", "id"],
        "score":   ["scores", "association_score"]
    }
)

17.12_evidence_data.json.gz is already downloaded


Feature Extraction: 5784597it [07:25, 12996.72it/s]


            target         disease  score
0  ENSG00000140859     EFO_0000519    1.0
1  ENSG00000197170     EFO_0000708    1.0
2  ENSG00000198650  Orphanet_28378    1.0
3  ENSG00000102468     EFO_0003761    1.0
4  ENSG00000043591     EFO_0000612    1.0


In [ ]:
!ls /root/workspace -al

total 5561440
drwxr-xr-x 2 root root       4096 Dec  7 06:00 .
drwx------ 1 root root       4096 Dec  7 03:38 ..
-rw-r--r-- 1 root root 5425463947 Dec  7 03:39 17.12_evidence_data.json.gz
-rw-r--r-- 1 root root  269427103 Dec  7 06:07 17.12_evidence_data.json.hdf


In [ ]:
!rm -rf /root/workspace/*.hdf
!ls /root/workspace -al

total 5298324
drwxr-xr-x 2 root root       4096 Dec  7 06:07 .
drwx------ 1 root root       4096 Dec  7 03:38 ..
-rw-r--r-- 1 root root 5425463947 Dec  7 03:39 17.12_evidence_data.json.gz
